学生のパフォーマンス

データセットの説明：

このデータは、ポルトガルの2008個の学校の中等教育における生徒の成績から取得したデータである。今回は数学にフォーカスしたデータを扱う。


特徴量：

sex:性別(F or M)

age:15~22の年齢

famsize:家族の規模(LE3:3人以下、GT3:３人以上)

Pstatus:親の同棲状況(T:同居、A:離れて)

Medu:母親の教育(0:なし、1:初等教育、2:5-9年生、3:中等教育、4:高等教育)

Fedu:父親の教育(0:なし、1:初等教育、2:5-9年生、3:中等教育、4:高等教育)

traveltime:自宅から学校までの移動時間(数値:1〜<15分、2〜15〜30分、3〜30分〜1時間、または4〜>1時間)

studytime:毎週の学習時間(数値:1 - <2時間、2 - 2 - 5時間、3 - 5から10時間、または4 - >10時間)

nursery:保育園に通った(バイナリ:はいまたはいいえ)

internet：-自宅でのインターネットアクセス(バイナリ:はいまたはいいえ)

romantic:ロマンチック-ロマンチックな関係(バイナリ: はいまたはいいえ)

famrel:家族関係の質 (数値: 1 - 非常に悪いから 5 - 非常に良い)

health:現在の健康状態 (数値: 1 - 非常に悪いから 5 - 非常に良い)

absences:欠席数 (数値: 0 から 93)

G3:最終成績

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

データの取得

In [6]:
#今回利用するデータは区切り文字が";"であるためsepに設定する
df = pd.read_csv('student-mat.csv',sep=";")

print(df)
print(df.shape)
print(df.columns)

    school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
390     MS   M   20       U     LE3       A     2     2  services  services   
391     MS   M   17       U     LE3       T     3     1  services  services   
392     MS   M   21       R     GT3       T     1     1     other     other   
393     MS   M   18       R     LE3       T     3     2  services     other   
394     MS   M   19       U     LE3       T     1     1     other   at_home   

     ... famrel freetime  goout  Dalc  Walc health 

データの整形

In [7]:
#入力(特徴量)と出力(目的変数)の分割
X = df.loc[:,['sex','age','famsize','Pstatus','Medu','Fedu','traveltime','studytime','nursery','internet','romantic','famrel','health','absences']]
le = LabelEncoder()
#ラベルエンコーディング

X['sex'] = le.fit_transform(df['sex'])
X['famsize'] = le.fit_transform(df['famsize'])
X['Pstatus'] = le.fit_transform(df['Pstatus'])
X['nursery'] = le.fit_transform(df['nursery'])
X['internet'] = le.fit_transform(df['internet'])
X['romantic'] = le.fit_transform(df['romantic'])

X = (X - X.mean())/X.std()

y = df['G3']

print(y)

0       6
1       6
2      10
3      15
4      10
       ..
390     9
391    16
392     7
393    10
394     9
Name: G3, Length: 395, dtype: int64


入力（特徴量）と出力（目的変数）の分離

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(276, 14) (119, 14) (276,) (119,)


モデルの構築

In [9]:
model = Sequential()
model.add(Conv1D(32, 3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

モデルの学習

In [10]:
epochs = 3000
batch_size = 50
history = model.fit(x=X_train, y=y_train, epochs=epochs, batch_size=batch_size, verbose=1 )

Epoch 1/3000
6/6 [==============================] - 3s 30ms/step - loss: 118.6139
Epoch 2/3000
6/6 [==============================] - 0s 4ms/step - loss: 85.3253
Epoch 3/3000
6/6 [==============================] - 0s 4ms/step - loss: 43.7285
Epoch 4/3000
6/6 [==============================] - 0s 4ms/step - loss: 30.8198
Epoch 5/3000
6/6 [==============================] - 0s 5ms/step - loss: 34.8093
Epoch 6/3000
6/6 [==============================] - 0s 4ms/step - loss: 27.9132
Epoch 7/3000
6/6 [==============================] - 0s 4ms/step - loss: 26.5674
Epoch 8/3000
6/6 [==============================] - 0s 3ms/step - loss: 26.1721
Epoch 9/3000
6/6 [==============================] - 0s 4ms/step - loss: 26.2173
Epoch 10/3000
6/6 [==============================] - 0s 4ms/step - loss: 26.3026
Epoch 11/3000
6/6 [==============================] - 0s 4ms/step - loss: 25.2270
Epoch 12/3000
6/6 [==============================] - 0s 3ms/step - loss: 25.0515
Epoch 13/3000
6/6 [================

モデルの予測

In [12]:
# 予測
y_pred = model.predict(X_test)
y_pred = np.reshape(y_pred, y_test.shape)

4/4 [==============================] - 0s 34ms/step


モデルの評価

ここでは平均二乗法とR2スコアを利用して評価する

In [13]:
# 評価（平均二乗誤差）
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 25.94239567584588


大体平均して５点ぐらい違う

In [14]:
# 評価（R2スコア）
r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

R2 Score: -0.1801619887874042


R2スコアの目安


<0.0 ターゲット変数の平均値を予測するより悪い

<0.1 悪い

<0.2 やや悪い

<0.3 悪い~普通

<0.5 普通

<0.7 普通~良い

<0.9 良い

>0.9 非常に良い


In [15]:
model.save('academy_cnn.h5')